In [1]:
import re

In [2]:
with open("./data_inputs/day15_input.txt") as f:
    input_raw = f.read()

sensors = {tuple(map(int, re.findall(r"-?\d+", line)[:2])): {"beacon": tuple(map(int, re.findall(r"-?\d+", line)[2:]))} for line in input_raw.split("\n")}

In [3]:
target_y = 2000000
target_x_ranges = []

for s in sensors:
    sensors[s]["dist"] = abs(s[0] - sensors[s]["beacon"][0]) + abs(s[1] - sensors[s]["beacon"][1])
    if abs(target_y - s[1]) <= sensors[s]["dist"]:
        target_dist = abs(target_y - s[1])
        target_x_rad = sensors[s]["dist"] - target_dist
        target_x_ranges.append((s[0] - target_x_rad, s[0] + target_x_rad))

In [4]:
# ---- Part 1 ----

def is_overlap(row):
    elf1_min, elf1_max = row[0]
    elf2_min, elf2_max = row[1]
    return (elf1_max >= elf2_min and elf2_max >= elf1_min)

is_any_overlap = True

while is_any_overlap:
    is_any_overlap = False

    for i, x_range1 in enumerate(target_x_ranges):
        for j, x_range2 in enumerate(target_x_ranges):
            if x_range1 == x_range2:
                continue
            elif is_overlap((x_range1, x_range2)):
                is_any_overlap = True
                new_range = (min(x_range1[0], x_range2[0]), max(x_range1[1], x_range2[1]))
                target_x_ranges.remove(x_range1)
                target_x_ranges.remove(x_range2)
                target_x_ranges.append(new_range)
                break

        if is_any_overlap:
            break

empty_positions = sum([x_range[1] - (x_range[0]-1) for x_range in target_x_ranges])

beacons_in_target = len(set([sensors[s]["beacon"] for s in list(sensors) if sensors[s]["beacon"][1] == target_y]))

print("Result 1:", empty_positions - beacons_in_target) 

Result 1: 5083287


In [5]:
# ---- Part 2 ----

get_freq = lambda x, y: x * 4_000_000 + y
dist = lambda p1, p2: abs(p1[0] - p2[0]) + abs(p1[1] - p2[1])

closest_sensors = []

for s1 in sensors:
    for s2 in sensors:
        sensors_dist = dist(s1, s2)
        if 0 <= abs((sensors[s1]["dist"] + sensors[s2]["dist"]) - (sensors_dist)) == 2:
            if (s2, s1) not in closest_sensors:
                closest_sensors.append((s1, s2))

((s1, s2), (s3, s4)) = closest_sensors

print(s1, s2)       # s1 is at the bottom left of s2

(3123411, 3392077) (3658837, 3026912)


In [6]:
d1 = sensors[s1]["dist"]
d3 = sensors[s3]["dist"]
d4 = sensors[s4]["dist"]

beacon = None
for i in range(d1 + 2):
    # scanning all points from the top right of the s1 limit colliding with the s2 limit
    p = (s1[0] + i, s1[1] - (d1-i+1))      
    if dist(p, s3) > d3 and dist(p, s4) > d4:
        beacon = p
        break

print("Result 2:", get_freq(beacon[0], beacon[1]))


Result 2: 13134039205729
